In [1]:
import pandas as pd
import numpy as np
import glob
import keras
import gensim
import string
from keras.models import load_model

Using TensorFlow backend.
unable to import 'smart_open.gcs', disabling that module


In [2]:
# return the predicted labels
def analyse(article):
    paragraphs = [par for par in article.split('\n') if str(par) != '' and len(par)>75]
    txt = [[word for word in par.lower().translate(string.punctuation).split()[:max_sentence_len]] for par in paragraphs]
    word_model = gensim.models.Word2Vec(txt, size=100, min_count=1, window=5, iter=100)
    x = np.zeros([len(txt), max_sentence_len], dtype=np.int32)
    for i, sentence in enumerate(txt):
        for t, word in enumerate(sentence):
            x[i, t] = word_model.wv.vocab[word].index
    sentiments = [round(pred[0],0) for pred in sentiment_model.predict(x)]
    frames = [round(pred[0],0) for pred in frame_model.predict(x)]
    eco = get_per(frames, 0.)
    hea = get_per(frames, 1.)
    env = get_per(frames, 2.)
    sen_score = get_per(sentiments, 1.)
    return sen_score, eco, hea, env

# percentage calculater
def get_per(l, n):
    try:
        tmp = round(l.count(n)/len(l)*100,1)
    except:
        tmp = 0
    return tmp

In [3]:
# load model
sentiment_model = load_model('models/sentiment/LSTM/lstm.h5')
frame_model = load_model('models/frame/LSTM/lstm.h5')

/home/roguehydra/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/roguehydra/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [5]:
# predict
files = glob.glob('articles/*')
max_sentence_len = 40
for file in files:
    article = open(file,'r').read()
    s_score, eco_score, hea_score, env_score = analyse(article)
    print('\n- - - - - - - - - - - - - - - -\n{}\n'.format(file))
    print('Sentiment: {}%'.format(s_score))
    print('\n- - - - - - - - - - - - - - - -\n')
    print('Economic frame: {}%\nHealth frame: {}%\nEnvironment frame: {}%'.format(eco_score, hea_score, env_score))
    print('\n- - - - - - - - - - - - - - - -\n')


- - - - - - - - - - - - - - - -
articles/dummy article 1

Sentiment: 69.0%

- - - - - - - - - - - - - - - -

Economic frame: 24.1%
Health frame: 51.7%
Environment frame: 24.1%

- - - - - - - - - - - - - - - -


- - - - - - - - - - - - - - - -
articles/dummy article 2

Sentiment: 50.0%

- - - - - - - - - - - - - - - -

Economic frame: 75.0%
Health frame: 25.0%
Environment frame: 0.0%

- - - - - - - - - - - - - - - -

